이번 과제는 Bert Model을 사용하여 BBC 뉴스 기사의 category를 분류해보는 과제입니다. clone coding을 하시되, 코드 주석을 line by line으로 꼼꼼하게 달아보시며 공부해보세요!

## 데이터 로드 및 탐색

In [2]:
%%capture
!pip install transformers

In [3]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
df = pd.read_csv('/content/drive/MyDrive/bbc-text.csv')

In [21]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [22]:
print(len(df))

2225


In [23]:
df.groupby('category').count()

,text
category,
business,510
entertainment,386
politics,417
sport,511
tech,401


## BertTokenizer

토크나이저로 pretrain된 BERT의 BertTokenizer를 갖고 옵니다. 여러 종류를 시도해보세요.

- bert-base-uncased : 108MB param, all lowercase
- bert-large-cased : 340MB param, both upper and lower
- bert-base-cased : 108MB param, multi language, both upper and lower


In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'business':0,
          'entertainment':1,
          'sport':2,
          'tech':3,
          'politics':4
          }

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Dataset

In [25]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

## Train & Evaluate BertClassifier

pretrain된 BertModel을 불러옵니다. 다른 간단한 층들도 같이 쌓아줍니다.

- bert-base-cased: 12-layer, 768-hidden, 12-self attention heads, 110M parameters. Trained on cased English text.


다른 종류들의 pretrianed model은 아래 링크에서 확인할 수 있습니다.

https://huggingface.co/transformers/v2.9.1/pretrained_models.html

In [26]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5): #BERT 모델 초기화

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5) #선형 레이어를 초기화-> BERT의 출력 크기(768)에서 분류하려는 클래스의 수(5)로 차원을 축소함
        self.relu = nn.ReLU() #ReLU 활성화 함수를 여기서 써줘야 함

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        #BERT 모델을 통해 입력 토큰들과 어텐션 마스크를 전달하여 출력 값을 얻기 때문에-> pre trained BERT 모델을 사용하여 문장의 표현을 인코딩
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        #linear 레이어를 통해 드롭아웃 출력을 전달하여 클래스 점수를 계산-> 입력 문장을 5개의 클래스 중 하나로 분류함
        final_layer = self.relu(linear_output)

        return final_layer #각 입력 문장에 대한 예측된 클래스 점수

In [27]:
def train(model, train_data, val_data, learning_rate, epochs):

    #훈련 데이터셋과 검증 데이터셋을 Dataset 객체로 변환함
    # (데이터셋을 데이터로더에 전달하기 위해)
    train, val = Dataset(train_data), Dataset(val_data)

    # 훈련 데이터로더를 생성함
    #  batch_size=2로 설정하고 데이터를 섞어(shuffle=True) 데이터셋에서 미니배치를 만듦
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)

    # 검증 데이터로더를 생성함
    # batch_size=2로 설정하고 데이터를 섞지 않음
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # 손실 함수를 설정: CrossEntropyLoss는 분류 문제에서 자주 사용되는 손실 함수
    criterion = nn.CrossEntropyLoss()

    # 옵티마이저를 설정 -> Adam 옵티마이저를 사용하여 모델의 파라미터를 학습률과 함께 업데이트함
    optimizer = Adam(model.parameters(), lr=learning_rate)

    # 모델과 손실 함수를 GPU로 이동함
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    # 학습을 지정된 에포크 수만큼 반복함
    for epoch_num in range(epochs):

        # 훈련 정확도와 손실의 총합을 초기화함
        total_acc_train = 0
        total_loss_train = 0

        # 훈련 데이터로더에서 배치를 반복하여 모델을 학습시킴
        for train_input, train_label in tqdm(train_dataloader):

            # 훈련 라벨을 디바이스에 맞게 이동시킴
            train_label = train_label.to(device)

            # 어텐션 마스크를 디바이스에 맞게 이동시킴
            mask = train_input['attention_mask'].to(device)

            # 입력 아이디를 디바이스에 맞게 이동시키고 차원을 맞추기 위해 squeeze 함
            input_id = train_input['input_ids'].squeeze(1).to(device)

            # 모델을 통해 출력을 얻음
            output = model(input_id, mask)

            # 배치 손실을 계산함
            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()

            # 예측 결과와 실제 라벨을 비교하여 정확도를 계산함
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            # 모델의 기울기를 초기화함
            model.zero_grad()

            # 역전파를 통해 기울기를 계산함
            batch_loss.backward()

            # 옵티마이저를 사용하여 모델의 파라미터를 업데이트함
            optimizer.step()

        # 검증 정확도와 손실의 총합을 초기화함
        total_acc_val = 0
        total_loss_val = 0

        # 검증 데이터에 대해 기울기를 계산하지 않도록 설정함
        with torch.no_grad():

            # 검증 데이터로더에서 배치를 반복하여 모델을 평가함
            for val_input, val_label in val_dataloader:

                # 검증 라벨을 디바이스에 맞게 이동시킴
                val_label = val_label.to(device)

                # 어텐션 마스크를 디바이스에 맞게 이동시킴
                mask = val_input['attention_mask'].to(device)

                # 입력 아이디를 디바이스에 맞게 이동시키고 차원을 맞추기 위해 squeeze 함
                input_id = val_input['input_ids'].squeeze(1).to(device)

                # 모델을 통해 출력을 얻음
                output = model(input_id, mask)

                # 배치 손실을 계산함
                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                # 예측 결과와 실제 라벨을 비교하여 정확도를 계산함
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        # 현재 에포크의 훈련 및 검증 손실과 정확도를 출력함
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [28]:
def evaluate(model, test_data):

    # 테스트 데이터셋을 Dataset 객체로 변환
    test = Dataset(test_data)

    # 테스트 데이터로더를 생성함->batch_size=2로 설정하고 데이터를 섞지 않음
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    # 테스트 정확도의 총합을 초기화 => 이 과정이 없다면 정확도를 계산할 수 없음.
    total_acc_test = 0


    #기울기 계산 비활성화 및 테스트 데이터 평가
    #테스트 데이터에 대해 기울기를 계산하지 않도록 설정함
    with torch.no_grad():

        # 테스트 데이터로더에서 배치를 반복하여 모델을 평가함
        #기울기를 계산하지 않고 테스트 데이터로더에서 배치를 반복하여 모델을 평가함. 이 과정이 없다면 평가 시 기울기가 계산되어 불필요한 메모리 사용이 발생함.
        for test_input, test_label in test_dataloader:

            # 테스트 라벨을 디바이스에 맞게 이동시킴
            test_label = test_label.to(device)

            # 어텐션 마스크를 디바이스에 맞게 이동시킴
            mask = test_input['attention_mask'].to(device)

            # 입력 아이디를 디바이스에 맞게 이동시키고 차원을 맞추기 위해 squeeze 함
            input_id = test_input['input_ids'].squeeze(1).to(device)

            #테스트 라벨과 입력 데이터를 디바이스에 맞게 이동시키고 차원을 맞추기 위해 squeeze 함수를 사용함.

            # 모델을 통해 출력을 얻음
            output = model(input_id, mask)

            # 예측 결과와 실제 라벨을 비교하여 정확도를 계산함
            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc

    # 테스트 데이터의 정확도를 출력함
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')


In [29]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])
#데이터프레임을 무작위로 섞고, 이를 80%, 10%, 10%의 비율로 훈련, 검증, 테스트 데이터로 나눔.
#np.split 함수는 지정된 인덱스 위치에서 데이터프레임을 나눔.

#각 데이터셋의 크기를 출력함->데이터셋이 예상대로 분할되었는지 확인함.
print(len(df_train),len(df_val), len(df_test))

1780 222 223


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [30]:
#data sample
print(df_train.head())
print(df_val.head())

#format
print(type(df_train))
print(type(df_val))

#첫번째 sample
print(df_train.iloc[0])
print(df_val.iloc[0])


           category                                               text
414        politics  brown and blair face new rift claims for the u...
420        business  small firms  hit by rising costs  rising fuel ...
1644  entertainment  spirit awards hail sideways the comedy sideway...
416            tech  microsoft releases patches microsoft has warne...
1232          sport  arsenal through on penalties arsenal win 4-2 o...
           category                                               text
1664          sport  blues slam blackburn over savage birmingham ha...
1012       politics  galloway targets  new labour  mp george gallow...
1871  entertainment  singer ferguson  facing eviction  three degree...
1150           tech  intel unveils laser breakthrough intel has sai...
790            tech  text message record smashed uk mobile owners c...
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
category                                             politics
text      

In [38]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer



def train(model, train_data, val_data, learning_rate, epochs):
    train, val = Dataset(train_data), Dataset(val_data)
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)

    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].to(device)

            # Shape check and adjustment
            if len(input_id.shape) > 2:
                input_id = input_id.squeeze(1)
            if len(mask.shape) > 2:
                mask = mask.squeeze(1)

            output = model(input_id, mask)

            batch_loss = criterion(output, train_label.long())
            total_loss_train += batch_loss.item()

            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()

        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():
            for val_input, val_label in val_dataloader:
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].to(device)

                # Shape check and adjustment
                if len(input_id.shape) > 2:
                    input_id = input_id.squeeze(1)
                if len(mask.shape) > 2:
                    mask = mask.squeeze(1)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label.long())
                total_loss_val += batch_loss.item()

                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc

        print(f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')



def evaluate(model, test_data):
    test = Dataset(test_data)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():
        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].to(device)

            # Shape check and adjustment
            if len(input_id.shape) > 2:
                input_id = input_id.squeeze(1)
            if len(mask.shape) > 2:
                mask = mask.squeeze(1)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [39]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42),
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1780 222 223


In [40]:
EPOCHS = 2 #EPOCH 수 늘려보기!
model = BertClassifier()
LR = 1e-6

train(model, df_train, df_val, LR, EPOCHS)

100%|██████████| 890/890 [03:11<00:00,  4.64it/s]


Epochs: 1 | Train Loss:  0.768 | Train Accuracy:  0.316 | Val Loss:  0.661 | Val Accuracy:  0.509


100%|██████████| 890/890 [03:10<00:00,  4.67it/s]


Epochs: 2 | Train Loss:  0.542 | Train Accuracy:  0.603 | Val Loss:  0.337 | Val Accuracy:  0.833


In [41]:
evaluate(model, df_test)

Test Accuracy:  0.816
